In [26]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import json
import os
import time
import datetime
import re
from dateutil.parser import parse as parsedate
from fake_useragent import UserAgent

# url = "https://www.ptt.cc/bbs/Stock/index.html"
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
session = requests.session()
session.post("https://www.ptt.cc/ask/over18", data={"yes": "yes"})
def get_hotboards():
    url = "https://www.ptt.cc/bbs/hotboards.html"
    r = session.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    boards = soup.select("a.board")
    return boards

def get_last_modified(url):
    r = requests.head(url)
    dt = parsedate(r.headers['Date'])
    last_modified_at = int(time.mktime(dt.timetuple()))*1000
    return last_modified_at


def remove_punctuation(line):
    rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5\u0020-\u002f\u003a-\u0040]")
    line = rule.sub('',line)
    return line

def get_crawltime():
    dtime = datetime.datetime.now()
    ans_time = round(time.mktime(dtime.timetuple()))
    return ans_time*1000

def ouput_board_page_articles_json(filename , res):
    if not os.path.exists("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2"):
        os.makedirs("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2")
    with open("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2/" + filename + ".json" , 'wb') as f:
        f.write(json.dumps(res, indent = 4, ensure_ascii = False).encode('utf-8'))



def get_unixtime(time_string):
    month_vector = {"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
    month = month_vector[time_string[4:7]]
    year = time_string[-4:]
    #print(len(time_string))
    day = time_string[8:10]
    day = day.replace(" ","")
    ptime = time_string[11:19]
    publish_time = year+"-"+str(month)+"-"+day+" "+ptime
    print(publish_time)
    unixtime = round(time.mktime(datetime.datetime.strptime(publish_time, '%Y-%m-%d %H:%M:%S').timetuple()))
    return unixtime*1000


def insert_links(data):
    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/links", headers=post_headers, json=data)
        print(response.json())
    except Exception as e:
        print(e)
    

def insert_category(data):
    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/categorylist", headers=post_headers, json=data)
        print(response.json())
    except Exception as e:
        print(e)
    
def get_content(article_url):
    ua = UserAgent(use_cache_server=False)
    ua_headers = {"User-Agent": ua.random}
    r = session.get(article_url,headers=ua_headers)
    soup = BeautifulSoup(r.text, "html.parser")
    if r.status_code==404:
        return
    a_tags = soup.find_all('a')
    extra_links=[]
    for tag in a_tags:
        url = tag.get("href")
        try:            
            if url.split(':')[0] == "https":
                domain = url.split('/')[2]
                if not domain == "www.ptt.cc":
                      extra_links.append(url)
        except Exception as e:
            print(e)
    if len(extra_links) > 0:
        insert_links(extra_links)
    try:
        article = {}
        article['language'] = "ZH_TW"
        article['crawled_at'] = get_crawltime()
        article['Author'] = soup.select('.article-meta-value')[0].text
        article['category'] = "PTT_"+article_url.split("/")[4]
        article['title'] = soup.select('.article-meta-value')[2].text
        article['publish_at'] = get_unixtime(soup.select('.article-meta-value')[3].text)
        article['link'] = article_url
        article['last_modified_at'] = get_last_modified(article_url)
        content = []
        article['contents'] = r.text
        
    except Exception as e:
        print(e)
        print("Analysis %s occur Error" %article_url )

    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/ptt", headers=post_headers
                                 , json=json.dumps(article))
        print(response.json())
    except Exception as e:
        print(e)

def get_all_href(url):
    r = session.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    results = soup.select("div.title")
    for item in results:
        a_item = item.select_one("a")
        title = item.text
        if a_item:
            print(title, 'https://www.ptt.cc'+ a_item.get('href'))
            get_content('https://www.ptt.cc'+  a_item.get('href'))
            
# urls = get_hotboards()
# for boards in urls:
#     url = "https://www.ptt.cc"+boards['href']
#     categorylist = {"term":"","description":""}
#     category = boards["href"].split("/")[2]
#     categorylist["term"] = "PTT_"+category
#     categorylist["description"] = "PTT_"+category
#     insert_category(categorylist)
#     for page in range(1,40):
#         r = session.get(url)
#         soup = BeautifulSoup(r.text,"html.parser")
#         btn = soup.select('div.btn-group > a')
#         up_page_href = btn[3]['href']
#         next_page_url = 'https://www.ptt.cc' + up_page_href
#         url = next_page_url
#         print(url)
#         get_all_href(url = url)

In [27]:
boards = "https://www.ptt.cc/bbs/DataScience/index.html"
url = boards
categorylist = {"term":"","description":""}
category = "DataScience"
categorylist["term"] = "PTT_"+category
categorylist["description"] = "PTT_"+category
insert_category(categorylist)
print(url)
r = session.get(url)
soup = BeautifulSoup(r.text,"html.parser")
btn = soup.select('div.btn-group > a')
up_page_href = btn[3]['href']
pages = ((up_page_href.split("/")[3]).split(".")[0]).split("index")[1]
total_page = int(pages)
for page in range(1,total_page):
    url = "https://www.ptt.cc/bbs/"+category+"/index"+str(page)+".html"
    print(url)
    get_all_href(url)

HTTPConnectionPool(host='fpgindexer2.widelab.org', port=8100): Max retries exceeded with url: /api/web/v1/users (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa6d9811ac8>: Failed to establish a new connection: [Errno -2] Name or service not known',))
https://www.ptt.cc/bbs/DataScience/index.html
https://www.ptt.cc/bbs/DataScience/index1.html

congratulations
 https://www.ptt.cc/bbs/DataScience/M.1518346764.A.9C6.html
2018-2-11 18:59:21
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[閒聊] 恭喜
 https://www.ptt.cc/bbs/DataScience/M.1518364906.A.1EF.html
2018-2-12 00:01:43
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[恭喜]  恭喜開板
 https://www.ptt.cc/bbs/DataScience/M.1518365452.A.7D9.html
2018-2-12 00:10:49
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[閒聊] 所以可以幫選股票嗎
 https://www.ptt.cc/bbs/DataScience/M.1518377331.A.BF7.html
20


[閒聊] 恭喜開板
 https://www.ptt.cc/bbs/DataScience/M.1519702554.A.F83.html
2018-2-27 11:35:51
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[閒聊] 恭喜開板
 https://www.ptt.cc/bbs/DataScience/M.1519704186.A.EC3.html
2018-2-27 12:03:00
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[問題] 手寫數字辨識問題
 https://www.ptt.cc/bbs/DataScience/M.1519705516.A.4F3.html
{'message': '操作成功', 'data': ['5f17afc747be3c762c5134de']}
2018-2-27 12:25:13
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[閒聊] 恭喜開版
 https://www.ptt.cc/bbs/DataScience/M.1519717573.A.B13.html
2018-2-27 15:46:11
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[問題] reinforcement learning跟控制系統差異？
 https://www.ptt.cc/bbs/DataScience/M.1519759252.A.41A.html
2018-2-28 03:20:49
{'message': '操作成功', 'data': {'insert': {'count': 0, '_ids': []}, 'update': {'count': 1}}}

[閒聊] 等好久了！恭

2018-3-17 10:09:31
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17afff69471e5ffc67efac']}, 'update': {'count': 0}}}

[問題] 資料中包含類別+數值型態建議怎麼處理
 https://www.ptt.cc/bbs/DataScience/M.1521265814.A.043.html
2018-3-17 13:50:11
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0005072eb3e1f530d1a']}, 'update': {'count': 0}}}

[問題] 有關sklearn中的GridSearchCV 的問題
 https://www.ptt.cc/bbs/DataScience/M.1521284731.A.181.html
2018-3-17 19:05:24
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0020c386074731d639b']}, 'update': {'count': 0}}}

Fw: [請益] Data Engineer門檻問題
 https://www.ptt.cc/bbs/DataScience/M.1521358576.A.EB4.html
{'message': '操作成功', 'data': ['5f17b0070bcb8674055f658b', '5f17b0070bcb8674055f658c', '5f17b0070bcb8674055f658d', '5f17b0070bcb8674055f658e']}
2018-3-18 15:36:14
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0120c386074731d639d']}, 'update': {'count': 0}}}

[公告] 板主選務事宜
 https://www.ptt.cc/bbs/DataScience/M.

2018-3-27 11:36:48
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b05e69471e5ffc67efbb']}, 'update': {'count': 0}}}
https://www.ptt.cc/bbs/DataScience/index6.html

Re: [問題] Validation set 到底在做什麼?
 https://www.ptt.cc/bbs/DataScience/M.1522135917.A.895.html
2018-3-27 15:31:55
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b06569471e5ffc67efbc']}, 'update': {'count': 0}}}

[問題] 現在的手寫辨識技術
 https://www.ptt.cc/bbs/DataScience/M.1522149931.A.8EA.html
2018-3-27 19:25:28
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b06620c9b02aa877e0c0']}, 'update': {'count': 0}}}

[問題] 機器學習--完形填空疑問
 https://www.ptt.cc/bbs/DataScience/M.1522165990.A.A57.html
{'message': '操作成功', 'data': ['5f17b06656b59770b30cb88b']}
2018-3-27 23:53:07
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b06720c9b02aa877e0c1']}, 'update': {'count': 0}}}

Re: [問題] Validation set 到底在做什麼?
 https://www.ptt.cc/bbs/DataScience/M.1522167114.A.3A7.html
2018-3-28 00:1

{'message': '操作成功', 'data': ['5f17b0a156b59770b30cb8a2']}
2018-4-13 01:47:50
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0a769471e5ffc67efcf']}, 'update': {'count': 0}}}

[問題] FPGA加速machine learning?
 https://www.ptt.cc/bbs/DataScience/M.1523630362.A.ED3.html
2018-4-13 22:39:20
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0a878802823c1723652']}, 'update': {'count': 0}}}

[討論] 機器學習如何做好version control???
 https://www.ptt.cc/bbs/DataScience/M.1523654101.A.7A3.html
{'message': '操作成功', 'data': ['5f17b0a969471e5ffc67efd0', '5f17b0a969471e5ffc67efd1']}
2018-4-14 05:14:58
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0a978802823c1723653']}, 'update': {'count': 0}}}

Re: [討論] 機器學習如何做好version control???
 https://www.ptt.cc/bbs/DataScience/M.1523666881.A.BC5.html
2018-4-14 08:47:58
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b0ab0bcb8674055f659d']}, 'update': {'count': 0}}}

Re: [討論] 機器學習如何做好version control???

...


[問題] 用傳統分類來幫助object detection
 https://www.ptt.cc/bbs/DataScience/M.1562979178.A.BC0.html
2019-7-13 08:52:56
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b45056b59770b30cb9d1']}, 'update': {'count': 0}}}

[問題] 想請教 Transfer Learning 的概念問題
 https://www.ptt.cc/bbs/DataScience/M.1563150712.A.7F3.html
2019-7-15 08:31:49
HTTPConnectionPool(host='fpgindexer2.widelab.org', port=8100): Max retries exceeded with url: /api/web/v1/users (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa6db7a9ef0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

[問題] tensorflow gradients 對多維output
 https://www.ptt.cc/bbs/DataScience/M.1563254421.A.E11.html
HTTPConnectionPool(host='fpgindexer2.widelab.org', port=8100): Max retries exceeded with url: /api/web/v1/links (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa6d960a2b0>: Failed to establish a new connection: [Errno -2] Name or service

2019-8-24 17:47:12
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4a60c386074731d66c3']}, 'update': {'count': 0}}}

[問題] 賭俠的電腦分析
 https://www.ptt.cc/bbs/DataScience/M.1566729706.A.E10.html
{'message': '操作成功', 'data': ['5f17b4a65072eb3e1f530e56', '5f17b4a65072eb3e1f530e57']}
2019-8-25 18:41:44
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4ac56b59770b30cb9dc']}, 'update': {'count': 0}}}

[問題] 時間序列預測問題
 https://www.ptt.cc/bbs/DataScience/M.1566902768.A.252.html
2019-8-27 18:46:06
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4ad0c386074731d66c4']}, 'update': {'count': 0}}}

[新聞] 微軟麻將AI來了，第一波人類高手已經被
 https://www.ptt.cc/bbs/DataScience/M.1567162885.A.35F.html
{'message': '操作成功', 'data': ['5f17b4ad5072eb3e1f530e59']}
2019-8-30 19:01:24
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4ae0c386074731d66c5']}, 'update': {'count': 0}}}

[心得] [Object Detection] RCNN 簡介
 https://www.ptt.cc/bbs/DataScience/M.1567184316


[問題] 2個文本model不同input可以合併train嗎?
 https://www.ptt.cc/bbs/DataScience/M.1574771347.A.BA0.html
{'message': '操作成功', 'data': ['5f17b4ee47be3c762c513716', '5f17b4ee47be3c762c513717', '5f17b4ee47be3c762c513718', '5f17b4ee47be3c762c513719', '5f17b4ee47be3c762c51371a', '5f17b4ee47be3c762c51371b']}
2019-11-26 20:29:05
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4ef5072eb3e1f530e7b']}, 'update': {'count': 0}}}

[徵求] 找人一起由淺入深練習經典資料集
 https://www.ptt.cc/bbs/DataScience/M.1574812652.A.2F9.html
{'message': '操作成功', 'data': ['5f17b4f069471e5ffc67f245']}
2019-11-27 07:57:29
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4f556b59770b30cba27']}, 'update': {'count': 0}}}

[問題] CNN影像分類問題(背景類)
 https://www.ptt.cc/bbs/DataScience/M.1574926914.A.450.html
{'message': '操作成功', 'data': ['5f17b4f69c00c005cd3e36ba']}
2019-11-28 15:41:51
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b4fc20c9b02aa877e253']}, 'update': {'count': 0}}}

[問題] 關於Linux 系統的安裝請益

2020-2-2 21:59:20
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b55347be3c762c513761']}, 'update': {'count': 0}}}

Fw: [資訊] 面對職場重新來過-勞動部職前訓練課程
 https://www.ptt.cc/bbs/DataScience/M.1580704266.A.B7F.html
{'message': '操作成功', 'data': ['5f17b5535072eb3e1f530e8d', '5f17b5535072eb3e1f530e8e']}
2020-2-3 12:31:05
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b55978802823c1723896']}, 'update': {'count': 0}}}

[問題] 研究方向迷惘
 https://www.ptt.cc/bbs/DataScience/M.1580870610.A.1D9.html
2020-2-5 10:43:28
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b55a0c386074731d67a7']}, 'update': {'count': 0}}}

[問題] 遷移學習如何不破壞在原問題上學習的成果
 https://www.ptt.cc/bbs/DataScience/M.1581004497.A.8F6.html
{'message': '操作成功', 'data': ['5f17b55b69471e5ffc67f33c', '5f17b55b69471e5ffc67f33d']}
2020-2-6 23:54:55
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b55b47be3c762c513774']}, 'update': {'count': 0}}}

[問題] 數據少時如何做文章分類
 https://www.ptt.cc/bbs/Da

2020-4-19 07:27:22
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b59b47be3c762c51377c']}, 'update': {'count': 0}}}

[問題] yolov3觀念請益
 https://www.ptt.cc/bbs/DataScience/M.1587306236.A.79E.html
2020-4-19 22:23:54
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b59d47be3c762c51377d']}, 'update': {'count': 0}}}
https://www.ptt.cc/bbs/DataScience/index30.html

[問題] lightgbm 的 plot_tree()
 https://www.ptt.cc/bbs/DataScience/M.1587477611.A.864.html
2020-4-21 22:00:09
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b59e0bcb8674055f6876']}, 'update': {'count': 0}}}

[問題] 尋找自然場景文字 dataset
 https://www.ptt.cc/bbs/DataScience/M.1588149023.A.9CF.html
2020-4-29 16:30:20
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f17b59f2403d9564c47dc96']}, 'update': {'count': 0}}}

[討論] 如何評價新出的Yolo v4.?
 https://www.ptt.cc/bbs/DataScience/M.1588203381.A.7C9.html
2020-4-30 07:36:19
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': [

In [23]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import json
import os
import time
import datetime
import re
from dateutil.parser import parse as parsedate
from fake_useragent import UserAgent

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
session = requests.session()
session.post("https://www.ptt.cc/ask/over18", data={"yes": "yes"})

#取得熱門看板
def get_hotboards():
    url = "https://www.ptt.cc/bbs/hotboards.html"
    r = session.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    boards = soup.select("a.board")
    return boards
#取得網頁header中的最後更新時間並轉成unix time
def get_last_modified(url):
    r = requests.head(url)
    dt = parsedate(r.headers['Date'])
    last_modified_at = int(time.mktime(dt.timetuple()))*1000
    return last_modified_at
#過濾特殊字符(目前用不到)
def remove_punctuation(line):
    rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5\u0020-\u002f\u003a-\u0040]")
    line = rule.sub('',line)
    return line
#取得爬取該網頁的時間並轉成unix time
def get_crawltime():
    dtime = datetime.datetime.now()
    ans_time = round(time.mktime(dtime.timetuple()))
    return ans_time*1000
#將爬取內容寫出json檔(沒用到)
def ouput_board_page_articles_json(filename , res):
    if not os.path.exists("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2"):
        os.makedirs("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2")
    with open("/ptt_crawler/PPT_Crawler/PPT_Crawl_Result2/" + filename + ".json" , 'wb') as f:
        f.write(json.dumps(res, indent = 4, ensure_ascii = False).encode('utf-8'))
#取得Ptt該篇貼文的發文時間並轉成unix time
def get_unixtime(time_string):
    month_vector = {"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
    month = month_vector[time_string[4:7]]
    year = time_string[-4:]
    #print(len(time_string))
    day = time_string[8:10]
    day = day.replace(" ","")
    ptime = time_string[11:19]
    publish_time = year+"-"+str(month)+"-"+day+" "+ptime
    print(publish_time)
    unixtime = round(time.mktime(datetime.datetime.strptime(publish_time, '%Y-%m-%d %H:%M:%S').timetuple()))
    return unixtime*1000

#上傳額外連結至server
def insert_links(data):
    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/links", headers=post_headers, json=data)
        print(response.json())
    except Exception as e:
        print(e)
    
#新增category
def insert_category(data):
    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/categorylist", headers=post_headers, json=data)
        print(response.json())
    except Exception as e:
        print(e)

#爬取該篇貼文    
def get_content(article_url):
    ua = UserAgent(use_cache_server=False)
    ua_headers = {"User-Agent": ua.random}
    r = session.get(article_url,headers=ua_headers)
    soup = BeautifulSoup(r.text, "html.parser")
    if r.status_code==404:
        return
    a_tags = soup.find_all('a')
    extra_links=[]#初始化額外連結陣列
    
    ###爬取所有URL(只抓https開頭，過濾掉local路徑)，如果domain不是ptt.cc就上傳###
    for tag in a_tags:
        url = tag.get("href")
        try:
            if url.split(':')[0] == "https":
                domain = url.split('/')[2]
                if not domain == "www.ptt.cc":
                      extra_links.append(url)
        except Exception as e:
            print(e)
    if len(extra_links) > 0:
        insert_links(extra_links)
    try:
        article = {}
        article['language'] = "ZH_TW"
        article['crawled_at'] = get_crawltime()
        article['Author'] = soup.select('.article-meta-value')[0].text
        article['category'] = "PTT_"+article_url.split("/")[4]
        article['title'] = soup.select('.article-meta-value')[2].text
        article['publish_at'] = get_unixtime(soup.select('.article-meta-value')[3].text)
        article['link'] = article_url
        article['last_modified_at'] = get_last_modified(article_url)
        content = []
        article['contents'] = r.text
        
    except Exception as e:
        print(e)
        print("Analysis %s occur Error" %article_url )

    try:
        token_data = {
            "account": "antUser6",
            "password":"antUser6@widelab"
        }
        token_response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/users", json=token_data).json()
        token=token_response['token']
        post_headers = { 'Authorization': 'Bearer '+token}
        response = requests.post("http://fpgindexer2.widelab.org:8100/api/web/v1/ptt", headers=post_headers
                                 , json=json.dumps(article))
        print(response.json())
        #將結果寫入logfile
        filename = article_url.split("/")[4]+"_"+(article_url.split("/")[5]).split(".")[1]
        with open("/ptt_crawler/PPT_Crawler/logfile.txt","a")as ptt_log:
            log_msg = str(get_crawltime()) + " " + filename +"\n"+str(response.json())+"\n"
            ptt_log.write(log_msg)
    except Exception as e:
        print(e)

#取得看板該頁的所有貼文連結並且呼叫get_content()
def get_all_href(url):
    r = session.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    results = soup.select("div.title")
    for item in results:
        a_item = item.select_one("a")
        title = item.text
        if a_item:
            print(title, 'https://www.ptt.cc'+ a_item.get('href'))
            get_content('https://www.ptt.cc'+  a_item.get('href'))

urls = get_hotboards() #取得熱門看板URL
for boards in urls:
    url = "https://www.ptt.cc"+boards['href']
    categorylist = {"term":"","description":""}
    category = boards["href"].split("/")[2]
    categorylist["term"] = "PTT_"+category
    categorylist["description"] = "PTT_"+category
    insert_category(categorylist)
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    btn = soup.select('div.btn-group > a')
    up_page_href = btn[3]['href']
    pages = ((up_page_href.split("/")[3]).split(".")[0]).split("index")[1] #該看板總頁數
    total_page = int(pages)
    for page in range(total_page-40,total_page):
        url = "https://www.ptt.cc/bbs/"+category+"/index"+str(page)+".html"
        print(url)
        get_all_href(url)#看板URL傳入get_all_href()

{'message': '操作失敗，名稱已存在或是資料有誤'}
https://www.ptt.cc/bbs/Gossiping/index.html
https://www.ptt.cc/bbs/Gossiping/index39374.html

Re: [問卦] 20萬鎂薪水在美國是什麼樣等級的生活
 https://www.ptt.cc/bbs/Gossiping/M.1594940945.A.4A5.html
{'message': '操作成功', 'data': ['5f114a4056b59770b30ae8ee', '5f114a4056b59770b30ae8ef']}
2020-7-17 07:09:03
{'name': 'Unauthorized', 'message': 'Your request was made with invalid credentials.', 'code': 0, 'status': 401, 'type': 'yii\\web\\UnauthorizedHttpException'}

[問卦] 台女跟國外的行為是不是不太一樣？
 https://www.ptt.cc/bbs/Gossiping/M.1594940954.A.E4C.html
2020-7-17 07:09:12
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f114a4b56b59770b30ae8f0']}, 'update': {'count': 0}}}

[問卦] 妳知道我在等妳嗎 妳如果真的在乎我
 https://www.ptt.cc/bbs/Gossiping/M.1594941142.A.21B.html
{'message': '操作成功', 'data': ['5f114a4b0bcb8674055d60df']}
2020-7-17 07:12:17
{'message': '操作成功', 'data': {'insert': {'count': 1, '_ids': ['5f114a5156b59770b30ae8f1']}, 'update': {'count': 0}}}

[問卦] 實體三倍券為什麼要用套塑膠套?
 https://w

KeyboardInterrupt: 